# Variational Quantum Classifier
Team 3 final VQC work

In [157]:
#import packages needed
import pandas as pd
import numpy as np

from qiskit import BasicAer
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.aqua.algorithms import VQC
from qiskit.aqua.components.optimizers import SPSA
from qiskit.circuit.library import TwoLocal, ZZFeatureMap
from qiskit.aqua.utils import split_dataset_to_data_and_labels, map_label_to_class_name

#set seed for entire notebook
seed=2000
aqua_globals.random_seed = seed

In [101]:
#load the data and take a sample of each class using a stratified approach
h1=pd.read_csv("harass1.csv")
h1 = h1.sample(100)
h2=pd.read_csv("harass2.csv")
h2 = h2.sample(300)
h3=pd.read_csv("harass3.csv")
h3 = h3.sample(500)

#split into training and test sets using a stratified approach on the response variable
msk = np.random.rand(len(h1)) < 0.8
train1 = h1[msk]
test1 = h1[~msk]
train1=np.array(train1.values.tolist())
test1=np.array(test1.values.tolist())

msk = np.random.rand(len(h2)) < 0.8
train2 = h2[msk]
test2 = h2[~msk]
train2=np.array(train2.values.tolist())
test2=np.array(test2.values.tolist())

msk = np.random.rand(len(h3)) < 0.8
train3 = h3[msk]
test3 = h3[~msk]
train3=np.array(train3.values.tolist())
test3=np.array(test3.values.tolist())

#compile training and testing sets
data_dict_train={"1":train1,"2":train2,"3":train3}
data_dict_test={"1":test1,"2":test2,"3":test3}

In [102]:
#see data structure of training set as a sanity check
data_dict_train

{'1': array([[3, 5, 3, 1, 1],
        [0, 5, 4, 1, 1],
        [0, 5, 3, 2, 2],
        [1, 4, 2, 2, 2],
        [0, 5, 4, 1, 1],
        [0, 3, 1, 1, 1],
        [2, 5, 3, 1, 1],
        [0, 3, 2, 1, 1],
        [0, 2, 3, 1, 1],
        [0, 3, 4, 1, 1],
        [0, 5, 5, 1, 2],
        [0, 1, 4, 1, 1],
        [1, 5, 3, 1, 1],
        [1, 3, 2, 2, 2],
        [0, 3, 4, 1, 1],
        [0, 5, 2, 1, 1],
        [0, 1, 3, 1, 1],
        [0, 5, 3, 1, 1],
        [2, 3, 2, 1, 1],
        [0, 4, 2, 1, 1],
        [0, 2, 2, 1, 2],
        [0, 5, 4, 1, 1],
        [2, 3, 4, 2, 2],
        [0, 5, 2, 1, 1],
        [0, 5, 4, 1, 1],
        [2, 5, 4, 1, 1],
        [0, 4, 3, 1, 1],
        [0, 4, 3, 1, 1],
        [4, 3, 3, 1, 1],
        [0, 4, 3, 2, 2],
        [1, 1, 3, 2, 2],
        [1, 2, 1, 1, 1],
        [2, 5, 4, 1, 1],
        [1, 2, 3, 1, 1],
        [2, 5, 3, 1, 1],
        [0, 5, 3, 1, 1],
        [1, 3, 3, 1, 1],
        [0, 2, 3, 1, 1],
        [3, 2, 3, 2, 1],
        [0, 5, 5, 3,

In [103]:
#run the VQC on the Quantum simulator 
feature_dim = 4

feature_map = ZZFeatureMap(feature_dimension=feature_dim, reps=2)
optimizer = SPSA(maxiter=40, c0=4.0, skip_calibration=True)
var_form = TwoLocal(feature_dim, ['ry', 'rz'], 'cz', reps=3)
vqc = VQC(optimizer, feature_map, var_form, data_dict_train, data_dict_test)

backend = BasicAer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=seed, seed_transpiler=seed)

result = vqc.run(quantum_instance)

#print testing accuracy
print(f'Testing success ratio: {result["testing_accuracy"]}')

Testing success ratio: 0.4226190476190476


In [147]:
#make probabilities and class predictions for each observation in the test set for exploration
probs1, labels1 = vqc.predict(data_dict_test['1'])
probs2, labels2 = vqc.predict(data_dict_test['2'])
probs3, labels3 = vqc.predict(data_dict_test['3'])

In [149]:
#verify testing accuracy manually
correct = np.unique(labels1, return_counts=True)[1][0]+np.unique(labels2, return_counts=True)[1][1]+np.unique(labels3, return_counts=True)[1][2]
length = len(labels1)+len(labels2)+len(labels3)
correct/length

0.4226190476190476

In [156]:
#qubit diagram for reference
print(var_form)

     ┌──────────┐┌──────────┐            ┌──────────┐┌───────────┐             »
q_0: ┤ RY(θ[0]) ├┤ RZ(θ[4]) ├─■──■─────■─┤ RY(θ[8]) ├┤ RZ(θ[12]) ├─────────────»
     ├──────────┤├──────────┤ │  │     │ └──────────┘└┬──────────┤┌───────────┐»
q_1: ┤ RY(θ[1]) ├┤ RZ(θ[5]) ├─■──┼──■──┼──────■───────┤ RY(θ[9]) ├┤ RZ(θ[13]) ├»
     ├──────────┤├──────────┤    │  │  │      │       └──────────┘├───────────┤»
q_2: ┤ RY(θ[2]) ├┤ RZ(θ[6]) ├────■──■──┼──────┼────────────■──────┤ RY(θ[10]) ├»
     ├──────────┤├──────────┤          │      │            │      ├───────────┤»
q_3: ┤ RY(θ[3]) ├┤ RZ(θ[7]) ├──────────■──────■────────────■──────┤ RY(θ[11]) ├»
     └──────────┘└──────────┘                                     └───────────┘»
«                           ┌───────────┐┌───────────┐             »
«q_0: ──────■───────■─────■─┤ RY(θ[16]) ├┤ RZ(θ[20]) ├─────────────»
«           │       │     │ └───────────┘├───────────┤┌───────────┐»
«q_1: ──────■───────┼──■──┼───────■──────┤ RY(θ[17]) ├┤ RZ(θ[21]